In [1]:
from kaggle_secrets import UserSecretsClient
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification
import requests

In [2]:
# Function to make predictions
def make_predictions(model, texts):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Tokenize the input texts
    tokenizer = AutoTokenizer.from_pretrained('roberta-base')
    inputs = tokenizer(texts, truncation=True, padding=True, return_tensors='pt')
    model.to(device)

    # Move inputs to the same device as the model
    device = model.device
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted labels
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predictions = torch.argmax(logits, dim=1).tolist()
    confidence_scores = probabilities.max(dim=1).values.tolist()

    return predictions, confidence_scores
